In [1]:
import pandas as pd
import numpy as np
import re


# 데이터 열기

In [2]:
df_movie = pd.read_csv('./data/x_train_valid.csv')
df_oscars = pd.read_csv('./data/oscars_2020_2025.csv')
df_tmbd_top = pd.read_csv('./data/tmbd_top10_2020_2025.csv')

## 데이터 마지막 확인

In [3]:
print("df_movie.columns:", df_movie.columns.tolist())
print("df_oscars.columns:", df_oscars.columns.tolist())
print("df_tmbd_top.columns:", df_tmbd_top.columns.tolist())

df_movie.columns: ['title', 'release_date', 'type', 'runtime', 'season_count', 'episode_count', 'genre', 'director', 'cast', 'country', 'language', 'production_company', 'synopsis', 'tags', 'imdb_rating', 'imdb_review_count', 'text_input', 'production_company_is_missing', 'director_is_missing', 'cast_is_missing']
df_oscars.columns: ['Ceremony', 'Year', 'Class', 'Category', 'Film', 'Name', 'Winner']
df_tmbd_top.columns: ['country', 'country_name', 'title', 'release_date', 'vote_average', 'vote_count']


In [4]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66300 entries, 0 to 66299
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          66300 non-null  object 
 1   release_date                   66300 non-null  object 
 2   type                           66300 non-null  object 
 3   runtime                        66300 non-null  float64
 4   season_count                   66300 non-null  float64
 5   episode_count                  66300 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       66300 non-null  object 
 8   cast                           66300 non-null  object 
 9   country                        66300 non-null  object 
 10  language                       66300 non-null  object 
 11  production_company             66300 non-null  object 
 12  synopsis                       61682 non-null 

In [5]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      714 non-null    object
 5   Name      715 non-null    object
 6   Winner    195 non-null    object
dtypes: int64(2), object(5)
memory usage: 42.2+ KB


In [6]:
df_tmbd_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       333 non-null    object 
 1   country_name  333 non-null    object 
 2   title         333 non-null    object 
 3   release_date  333 non-null    object 
 4   vote_average  333 non-null    float64
 5   vote_count    333 non-null    int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 15.7+ KB


## 나머지 결측치 전처리

###  df_movie

In [7]:
df_movie['release_date'] = pd.to_datetime(df_movie['release_date']).dt.year.astype(str)

In [8]:
df_movie[df_movie['genre'].isna()]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,language,production_company,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing
4,12,2020,movie,15.0,1.0,1.0,NaN,Salih Hashwan,unknown,SA,ar,unknown,not there yet,"short film, short film",5.481,53,not there yet,1,0,1
628,Fight the Dragon: Big Breasts Game of Death,2020,movie,71.0,1.0,1.0,NaN,Hirohisa Sasaki,"Marina Yuzuki, Karina Nishida, Nina Nishimura,...",JP,ja,unknown,"The three sisters Satomi (Marina Yuzuki), Kaor...",short film,0.000,0,"The three sisters Satomi (Marina Yuzuki), Kaor...",1,0,0
659,From...,2020,movie,10.0,1.0,1.0,NaN,Sepideh Yadegar,unknown,US,en,unknown,This visual and musical feast sets the table f...,"romance, coming-of-age, rom-com, musical, adve...",7.500,2,This visual and musical feast sets the table f...,1,0,1
811,Zombie Wars,2020,movie,79.0,1.0,1.0,NaN,"Jason Wright, Kenneth Kjeldsen, Stefan Androv ...","Rickey Bird Jr., Laurence R. Harvey, Megan Loc...",GB,en,unknown,Anthology feature with short zombie films from...,"horror, adventure, survival",1.000,1,Anthology feature with short zombie films from...,1,0,0
815,Dancing Mita,2020,movie,104.0,1.0,1.0,NaN,Toshimitsu Iizuka,"Amane Okayama, Konatsu Kato, Yuichi Nakamura, ...",JP,ja,unknown,Mita (Amane Okayama) wanted to become a videog...,"family, psychological, superhero, teen",7.300,3,Mita (Amane Okayama) wanted to become a videog...,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66249,Victim Ya Victor,2025,tv,23.0,1.0,5.0,NaN,unknown,Rahul Dua,IN,hi,unknown,"""Victim ya Victor"" is a wild game show where f...","documentary, trending-theme",0.000,0,"""Victim ya Victor"" is a wild game show where f...",1,1,0
66251,De Grannies van Amsterdam,2025,tv,45.0,1.0,10.0,NaN,unknown,"Lorraine, Mug, Joanita, Tonneke, Jocelyn",NL,nl,unknown,In De Grannies van Amsterdam the cameras follo...,"teen, feel-good, romance, family, rom-com, sta...",0.000,0,In De Grannies van Amsterdam the cameras follo...,1,1,0
66256,Brandmerk: Kind van een Gedetineerde,2025,tv,35.0,1.0,6.0,NaN,unknown,Menen Seijkens,NL,nl,unknown,"In the Netherlands, at least 25,000 children h...",family,0.000,0,"In the Netherlands, at least 25,000 children h...",1,1,0
66258,Vicky Pattison: My Deepfake Sex Tape,2025,tv,47.0,1.0,1.0,NaN,unknown,Vicky Pattison,GB,en,unknown,Vicky boldly immerses herself in the world of ...,"historical, noir",0.000,0,Vicky boldly immerses herself in the world of ...,1,1,0


- genre nan 데이터 제거

In [9]:
df_movie = df_movie.dropna(subset=["genre"])

- synopsis nan은 unknown처리

In [10]:
df_movie["synopsis"] = df_movie["synopsis"].fillna("unknown")

In [11]:
df_movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 63278 entries, 0 to 66299
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          63278 non-null  object 
 1   release_date                   63278 non-null  object 
 2   type                           63278 non-null  object 
 3   runtime                        63278 non-null  float64
 4   season_count                   63278 non-null  float64
 5   episode_count                  63278 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       63278 non-null  object 
 8   cast                           63278 non-null  object 
 9   country                        63278 non-null  object 
 10  language                       63278 non-null  object 
 11  production_company             63278 non-null  object 
 12  synopsis                       63278 non-null  obje

## df_oscars

In [12]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,NaN
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,NaN
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,NaN
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,NaN


In [13]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      714 non-null    object
 5   Name      715 non-null    object
 6   Winner    195 non-null    object
dtypes: int64(2), object(5)
memory usage: 42.2+ KB


In [14]:
df_oscars["Film"] = df_oscars["Film"].fillna("unknown")
df_oscars["Film"] = df_oscars["Film"].fillna("unknown")

In [15]:
df_oscars["Winner"] = df_oscars["Winner"].astype(str).str.lower() == "true"

In [16]:
df_oscars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Ceremony  770 non-null    int64 
 1   Year      770 non-null    int64 
 2   Class     770 non-null    object
 3   Category  770 non-null    object
 4   Film      770 non-null    object
 5   Name      715 non-null    object
 6   Winner    770 non-null    bool  
dtypes: bool(1), int64(2), object(4)
memory usage: 37.0+ KB


## df_tmbd_top

In [17]:
df_tmbd_top.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   country       333 non-null    object 
 1   country_name  333 non-null    object 
 2   title         333 non-null    object 
 3   release_date  333 non-null    object 
 4   vote_average  333 non-null    float64
 5   vote_count    333 non-null    int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 15.7+ KB


In [18]:
df_tmbd_top['release_date'] = pd.to_datetime(df_tmbd_top['release_date']).dt.year.astype(str)
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020,8.500,165
1,US,United States,Gabriel's Inferno,2020,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020,8.400,83


# df_movie

In [19]:
df_movie.describe()

,runtime,season_count,episode_count,imdb_rating,imdb_review_count,production_company_is_missing,director_is_missing,cast_is_missing
count,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000,63278.000000
mean,71.535257,1.083441,5.949414,4.313007,47.763931,0.263551,0.159597,0.046746
std,50.508485,0.553268,22.614478,3.301194,352.550047,0.440562,0.366235,0.211096
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80.000000,1.000000,1.000000,5.500000,2.000000,0.000000,0.000000,0.000000
75%,98.000000,1.000000,1.000000,7.000000,9.000000,1.000000,0.000000,0.000000
max,6250.000000,62.000000,1204.000000,10.000000,20819.000000,1.000000,1.000000,1.000000


<strong>- imdb_rating * imdb_review_count 하여 상위 20%에 1 부여</strong>

In [20]:
df_movie["imdb_score"] = df_movie["imdb_rating"] * np.log1p(df_movie["imdb_review_count"])
threshold = df_movie["imdb_score"].quantile(0.8)

df_movie["success_imdb"] = (df_movie["imdb_score"] >= threshold).astype(int)

In [21]:
df_movie["success_imdb"].value_counts()

success_imdb
0    50622
1    12656
Name: count, dtype: int64

In [22]:
df_movie["success_imdb"].value_counts(normalize=True)

success_imdb
0    0.799994
1    0.200006
Name: proportion, dtype: float64

In [23]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,success_imdb
0,Stepmom's Desire,2020,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,1
1,Pizza Dare 1,2020,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0
2,Bosomy Mom,2020,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0
3,Sonic the Hedgehog,2020,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,1
5,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,"Tanjiro Kamado, joined with Inosuke Hashibira,...","fight, magic, supernatural, psychology, gore, ...",8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,1


# concat oscars


| 클래스                                                 | 의미            | 점수   |
| --------------------------------------------------- | ------------- | ---- |
| 수상 (`Winner == "True"`)                             | 실제 오스카 수상작/인물 | `2`점 |
| 후보 (`Class == "Nomination"` and `Winner != "True"`) | 수상은 못 했지만 후보작 | `1`점 |
| 없음                                                  | 오스카 언급 안 됨    | `0`점 |

In [24]:
def assign_award_score(row):
    if str(row["Winner"]).strip().lower() == "true":
        return 2
    elif str(row["Class"]).strip().lower() == "nomination":
        return 1
    else:
        return 1
    
df_oscars["award_score"] = df_oscars.apply(assign_award_score, axis=1)

In [25]:
df_oscars["award_score"].value_counts()

award_score
1    575
2    195
Name: count, dtype: int64

In [26]:
df_oscars["award_score"].value_counts(normalize=True)

award_score
1    0.746753
2    0.253247
Name: proportion, dtype: float64

In [27]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner,award_score
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,False,1
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,False,1
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True,2
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,False,1
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,False,1


# Top 10

In [28]:
df_tmbd_top.head()

,country,country_name,title,release_date,vote_average,vote_count
0,US,United States,Folklore: The Long Pond Studio Sessions,2020,8.500,165
1,US,United States,Gabriel's Inferno,2020,8.411,2423
2,US,United States,Gabriel's Inferno: Part II,2020,8.400,1522
3,US,United States,Gabriel's Inferno: Part III,2020,8.400,1058
4,US,United States,RBD: Ser o Parecer (En Vivo),2020,8.400,83


In [29]:
# 정렬 기준이 필요해요: vote_average or vote_count 기준
df_tmbd_top = df_tmbd_top.sort_values(["country", "vote_average", "vote_count"], ascending=[True, False, False])

# 기본 점수는 1.0
df_tmbd_top["top_score"] = 1.0

# 상위 20%: 나라별 상위 2편에게 +0.1점
df_tmbd_top["bonus_score"] = (
    df_tmbd_top.groupby("country").cumcount() < 2  # 상위 0, 1번 인덱스만
).astype(float) * 0.1

# 최종 점수
df_tmbd_top["top_score"] += df_tmbd_top["bonus_score"]

In [30]:
df_tmbd_top["title_lower"] = df_tmbd_top["title"].str.lower()

top_scores = df_tmbd_top.groupby("title_lower")["top_score"].sum().reset_index()

In [31]:
df_tmbd_top = df_tmbd_top.drop(columns=["bonus_score", "title_lower"])
df_tmbd_top

,country,country_name,title,release_date,vote_average,vote_count,top_score
100,CN,China,Green Snake,2021,8.100,327,1.1
101,CN,China,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,China,Ne Zha 2,2025,8.042,179,1.0
220,CN,China,Hachiko,2023,7.879,253,1.0
102,CN,China,B for Busy,2021,7.700,54,1.0
...,...,...,...,...,...,...,...
306,US,United States,The Life of Chuck,2025,7.737,59,1.0
305,US,United States,The Gorge,2025,7.711,2706,1.0
307,US,United States,The Unbreakable Boy,2025,7.630,73,1.0
308,US,United States,Sinners,2025,7.549,1563,1.0


In [32]:
df_tmbd_top.columns

Index(['country', 'country_name', 'title', 'release_date', 'vote_average',
       'vote_count', 'top_score'],
      dtype='object')

In [33]:
df_tmbd_top = df_tmbd_top[['country', 'title', 'release_date', 'vote_average', 'vote_count', 'top_score']]
df_tmbd_top.head()

,country,title,release_date,vote_average,vote_count,top_score
100,CN,Green Snake,2021,8.100,327,1.1
101,CN,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,Ne Zha 2,2025,8.042,179,1.0
220,CN,Hachiko,2023,7.879,253,1.0
102,CN,B for Busy,2021,7.700,54,1.0


# 문자열 칼럼 데이터 통일

In [34]:
print(df_movie.info())
print(df_oscars.info())
print(df_tmbd_top.info())

<class 'pandas.core.frame.DataFrame'>
Index: 63278 entries, 0 to 66299
Data columns (total 22 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   title                          63278 non-null  object 
 1   release_date                   63278 non-null  object 
 2   type                           63278 non-null  object 
 3   runtime                        63278 non-null  float64
 4   season_count                   63278 non-null  float64
 5   episode_count                  63278 non-null  float64
 6   genre                          63278 non-null  object 
 7   director                       63278 non-null  object 
 8   cast                           63278 non-null  object 
 9   country                        63278 non-null  object 
 10  language                       63278 non-null  object 
 11  production_company             63278 non-null  object 
 12  synopsis                       63278 non-null  obje

In [35]:
df_movie.head()

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,success_imdb
0,Stepmom's Desire,2020,movie,78.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Lee Soo, Tae Hee, James, Jung In",KR,...,"Sang-jin, who was envious of her neighbor afte...",softcore,7.150,20,"Sang-jin, who was envious of her neighbor afte...",1,0,0,21.768335,1
1,Pizza Dare 1,2020,movie,68.0,1.0,1.0,Romance,Kong Ja-kwan,"Yoo Jung, Yoon So-jung, Jin Si-ah, Baek Yoon-j...",KR,...,"Seok, who wants to try perverted sex, pesters ...",softcore,6.222,9,"Seok, who wants to try perverted sex, pesters ...",1,0,0,14.326684,0
2,Bosomy Mom,2020,movie,69.0,1.0,1.0,"Drama, Romance",Lee Dong-Joon,"Yoo Jung, Han Yi-seul, James, Shin Yeon-woo",KR,...,Da-hee gets married to Min-soo who is older th...,softcore,5.900,11,Da-hee gets married to Min-soo who is older th...,1,0,0,14.660949,0
3,Sonic the Hedgehog,2020,movie,99.0,1.0,1.0,"Action, Science Fiction, Comedy, Family",Jeff Fowler,"Ben Schwartz, James Marsden, Tika Sumpter, Jim...",US,...,"Powered with incredible speed, Sonic The Hedge...","friendship, video game, san francisco, califor...",7.307,9955,"Powered with incredible speed, Sonic The Hedge...",0,0,0,67.267735,1
5,Demon Slayer -Kimetsu no Yaiba- The Movie: Mug...,2020,movie,117.0,1.0,1.0,"Animation, Action, Fantasy, Thriller",Haruo Sotozaki,"Natsuki Hanae, Akari Kito, Hiro Shimono, Yoshi...",JP,...,"Tanjiro Kamado, joined with Inosuke Hashibira,...","fight, magic, supernatural, psychology, gore, ...",8.209,4103,"Tanjiro Kamado, joined with Inosuke Hashibira,...",0,0,0,68.296560,1


In [36]:
df_oscars.head()

,Ceremony,Year,Class,Category,Film,Name,Winner,award_score
0,93,2020,Acting,ACTOR IN A LEADING ROLE,Sound of Metal,Riz Ahmed,False,1
1,93,2020,Acting,ACTOR IN A LEADING ROLE,Ma Rainey's Black Bottom,Chadwick Boseman,False,1
2,93,2020,Acting,ACTOR IN A LEADING ROLE,The Father,Anthony Hopkins,True,2
3,93,2020,Acting,ACTOR IN A LEADING ROLE,Mank,Gary Oldman,False,1
4,93,2020,Acting,ACTOR IN A LEADING ROLE,Minari,Steven Yeun,False,1


In [37]:
df_tmbd_top.head()

,country,title,release_date,vote_average,vote_count,top_score
100,CN,Green Snake,2021,8.100,327,1.1
101,CN,New Gods: Nezha Reborn,2021,8.095,462,1.1
325,CN,Ne Zha 2,2025,8.042,179,1.0
220,CN,Hachiko,2023,7.879,253,1.0
102,CN,B for Busy,2021,7.700,54,1.0


In [39]:
df_movie[df_movie['title']=="The Father"]

,title,release_date,type,runtime,season_count,episode_count,genre,director,cast,country,...,synopsis,tags,imdb_rating,imdb_review_count,text_input,production_company_is_missing,director_is_missing,cast_is_missing,imdb_score,success_imdb
36,The Father,2020,movie,97.0,1.0,1.0,Drama,Florian Zeller,"Anthony Hopkins, Olivia Colman, Mark Gatiss, O...",GB,...,A man refuses all assistance from his daughter...,"london, england, loss of sense of reality, fla...",8.113,3343,A man refuses all assistance from his daughter...,0,0,0,65.836370,1
3282,The Father,2020,movie,90.0,1.0,1.0,Drama,"Kristina Grozeva, Petar Valchanov","Ivan Barnev, Ivan Savov, Tanya Shahova, Hristo...",BG,...,Vasil has just lost his long-time partner in l...,"drama, historical",5.700,10,Vasil has just lost his long-time partner in l...,0,0,0,13.668003,0
63721,The Father,2025,movie,15.0,1.0,1.0,Drama,Mikhail Emelyanov,"Vyacheslav Shchenin, Yuliya Mandriko, Natalia ...",RU,...,"Mikhail, who abandoned his wife and daughter i...","coming-of-age, drama, romance",0.000,0,"Mikhail, who abandoned his wife and daughter i...",0,0,0,0.000000,0


# ✅ 1. 문자열 정제 함수 정의
def clean_text(val):
    if not isinstance(val, str):
        return val
    val = val.lower()
    val = val.strip()
    val = val.replace("’", "'").replace("‘", "'")
    val = val.replace("–", "-").replace("—", "-")
    val = re.sub(r"\s+", " ", val)  # 다중 공백 → 하나
    return val

# ✅ 2. 특정 DataFrame에 문자열 컬럼 정제 적용하는 함수
def clean_object_columns(df):
    df = df.copy()
    for col in df.select_dtypes(include="object").columns:
        df[col] = df[col].apply(clean_text)
    return df

# ✅ 3. 모든 주요 데이터프레임에 적용
df_movie = clean_object_columns(df_movie)
df_oscars = clean_object_columns(df_oscars)
df_tmbd_top = clean_object_columns(df_tmbd_top)


print(df_movie[["title", "director", "cast"]].sample(3))
print(df_oscars[["Film", "Name"]].sample(3))
print(df_tmbd_top[["title", "country"]].sample(3))